# โจทย์: Automate SQL ได้ไหม?


## SQLite with Python

In [1]:
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

In [3]:
engine

Engine(sqlite:///mysqlite01.db)

In [4]:

# สร้างตาราง products ด้วย engine.execute
with engine.connect() as conn:
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL NOT NULL,
        category TEXT
    )
    '''))

    # สร้างตาราง orders
    conn.execute(text('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER,
        order_date TEXT
    )
    '''))

In [5]:
with engine.connect() as conn:
    # เพิ่มข้อมูลในตาราง products
    conn.execute(text('''
        INSERT INTO products (name, price, category)
        VALUES ('iPhone 14', 35900, 'Electronics')
    '''))
    
    # เพิ่มข้อมูลในตาราง orders 
    conn.execute(text('''
        INSERT INTO orders (product_id, quantity, order_date)
        VALUES (1, 2, '2023-10-20')
    '''))
    
    conn.commit()

## Faker

In [6]:
from faker import Faker

# สร้าง Faker object สำหรับภาษาไทย
fake = Faker()

fake_data = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000),
    "category": fake.random_element(
        elements=("Electronics", "Clothing", "Food", "Books")
    ),
}
fake_data

{'name': 'AirPods', 'price': 647, 'category': 'Books'}

## Faker + SQLite

In [12]:
# สร้างข้อมูลปลอมด้วย Faker
fake_product = {
    "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
    "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
}
fake_product

{'name': 'iPhone', 'price': 46210, 'category': 'Electronics'}

In [13]:
# เพิ่มข้อมูลลงใน SQLite
with engine.connect() as conn:
    conn.execute(text('''
        INSERT INTO products (name, price, category)
        VALUES (:name, :price, :category)
    '''), fake_product)
    conn.commit()

## Pandas

In [15]:
import pandas as pd

table (dataframe) = index + column + data

|index|column1|column2|column3|
|---|---|---|---|
|0|data01|data02|data03|
|1|data11|data12|data13|

In [16]:
# สร้าง DataFrame จากข้อมูลพื้นฐาน
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

,name,age,city
0,สมชาย,25,กรุงเทพ
1,สมหญิง,30,เชียงใหม่
2,สมศรี,35,ภูเก็ต


In [17]:
demo = {
    "player": ["LeBron James", "Kevin Durant", "Stephen Curry"],
    "team": ["Lakers", "Nets", "Warriors"],
}
demo_df = pd.DataFrame(demo)
demo_df

,player,team
0,LeBron James,Lakers
1,Kevin Durant,Nets
2,Stephen Curry,Warriors


In [22]:
name_series = df["name"]
name_series

0     สมชาย
1    สมหญิง
2     สมศรี
Name: name, dtype: object

In [23]:
age_series = df["age"]
age_series

0    25
1    30
2    35
Name: age, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3 non-null      object
 1   age     3 non-null      int64 
 2   city    3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


In [25]:
# แสดงข้อมูลเบื้องต้นของ DataFrame
df.describe()

,age
count,3.0
mean,30.0
std,5.0
min,25.0
25%,27.5
50%,30.0
75%,32.5
max,35.0


In [ ]:
# Python สามารถนำไปวิเคราะห์ข้อมูลต่อได้

## Pandas + SQLite

In [26]:
data = {
    'name': ['สมชาย', 'สมหญิง', 'สมศรี'],
    'age': [25, 30, 35],
    'city': ['กรุงเทพ', 'เชียงใหม่', 'ภูเก็ต']
}
df = pd.DataFrame(data)
df

,name,age,city
0,สมชาย,25,กรุงเทพ
1,สมหญิง,30,เชียงใหม่
2,สมศรี,35,ภูเก็ต


In [27]:
df.to_sql('people', engine, if_exists='append', index=False)

3

## Pandas + SQLite + Faker

In [28]:
# สร้างข้อมูลปลอมด้วย Faker
products = []   
for _ in range(100):
    product = {
        "name": fake.random_element(elements=("iPhone", "iPad", "MacBook", "AirPods")),
    "price": fake.random_int(min=100, max=50000), 
        "category": fake.random_element(elements=("Electronics", "Clothing", "Food", "Books"))
    }
    products.append(product)

# สร้าง DataFrame สำหรับสินค้า
df_products = pd.DataFrame(products)
df_products

,name,price,category
0,iPhone,29890,Electronics
1,iPhone,8568,Books
2,iPhone,19953,Food
3,iPad,33782,Food
4,MacBook,28397,Food
...,...,...,...
95,iPhone,23185,Books
96,iPhone,26386,Food
97,MacBook,45343,Electronics
98,MacBook,10401,Clothing


In [29]:
df_products.to_sql('products', engine, if_exists='append', index=False)

100

## Challege: สร้างข้อมูลลูกค้าโดยใช้ Faker และ Pandas และเพิ่มลงใน SQLite ทุก 5 วินาที

In [30]:
# faker to df
import faker
import pandas as pd
import time
import pendulum

# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()
def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers
customers = generate_customers(2)
customer_df = pd.DataFrame(customers)
customer_df

,customer_id,name,email,phone,address,segment,join_date,loyalty_points
0,8d78ea32-9c18-4db7-87d8-d460e8eece99,Shawn Cox,cbrown@example.com,(436)350-8558x55775,"505 Sandoval Isle\nKimberlyton, MA 02703",In-Store,2024-11-30 10:35:45.984406+07:00,208
1,70cc96ab-a1be-4681-8390-452241371a55,Shannon Sutton,kevinreynolds@example.com,777.524.5014,"8452 Wilson Knoll Apt. 612\nPort Kennethside, ...",In-Store,2024-11-30 10:35:45.985209+07:00,43


In [31]:
for _ in range(100):
    customers = generate_customers(1)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    time.sleep(5)

In [ ]:
# df to sqlite

# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

## Final App

In [ ]:
import faker
import pandas as pd
import time
# เปลี่ยนจาก sqlite3 เป็น sqlalchemy
from sqlalchemy import create_engine, text

# สร้าง engine แทนการใช้ connect โดยตรง
engine = create_engine('sqlite:///mysqlite01.db')
# สร้าง Faker object สำหรับสร้างข้อมูลปลอม
fake = faker.Faker()

def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):  # สร้าง 5 รายการ
        customer = {
            'customer_id': fake.uuid4(),
            'name': fake.name(),
            'email': fake.email(), 
            'phone': fake.phone_number(),
            'address': fake.address(),
            'segment': fake.random_element(['Retail', 'Wholesale', 'Online', 'In-Store']),
            'join_date': pendulum.now('Asia/Bangkok'),
            'loyalty_points': fake.random_int(min=0, max=1000)
        }
        customers.append(customer)
    return customers

# สร้างข้อมูลลูกค้า
while True:
    customers = generate_customers(5)
    customer_df = pd.DataFrame(customers)
    customer_df.to_sql('customers', engine, if_exists='append', index=False)
    print(f"Created {len(customers)} customers at {time.strftime('%H:%M:%S')}")
    time.sleep(5)

In [ ]:
# Python สามารถ generate data เข้า database แบบ realtime ได้แล้ว

## Congratulation!!